In [1]:
import ase
import numpy as np
from numpy.linalg import norm, svd
from ase.visualize import view
import struct_statistics as stats
import matplotlib.pyplot as p
import soaputils as su
#%matplotlib

In [2]:
# generate and read the two structures used in this notebook
struct_ran = su.gen_struct(70, seed=50)
struct_opt = ase.io.read('res_structs/x0_test/batch0.cfg')

In [ ]:
# calculate SOAP-parameters and matrices, only works on linux systems, 
# so the matrices were pre-calculated can be loaded in the cell below
import soaplite
import genBasis
rCut = 10.0
NradBas = 5
Lmax = 5
myAlphas, myBetas = genBasis.getBasisFunc(rCut, NradBas)

at_ran = soaplite.get_periodic_soap_structure(struct_ran, myAlphas, myBetas, rCut, NradBas, Lmax)
mat_opt = soaplite.get_periodic_soap_structure(struct_opt, myAlphas, myBetas, rCut, NradBas, Lmax)
np.save("res_data/mat_ran",mat_ran)
np.save("res_data/mat_opt",mat_opt)

In [3]:
# load matrices from storage
mat_ran = np.load("res_data/mat_ran.npy")
mat_opt = np.load("res_data/mat_opt.npy")

In [4]:
# show random structure
view(struct_ran)

In [5]:
# show optimised structure
view(struct_opt)

In [14]:
# show SVD plots
s_ran = svd(mat_ran.transpose(), full_matrices=False, compute_uv=False)
s_opt = svd(mat_opt.transpose(), full_matrices=False, compute_uv=False)

p.semilogy(s_ran)
p.semilogy(s_opt)
p.xlabel('$i$')
p.ylabel('Singular value $\sigma_i$')
p.legend(['random structure','optimised structure'])



In [19]:
%matplotlib qt5
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

np.random.seed(19680801)
Nr = 1
Nc = 2
cmap = "inferno"

fig, axs = plt.subplots(Nr, Nc)
#fig.suptitle('Multiple images')

count = 0
images = []
data = mat_ran
images.append(axs[0].imshow(data, cmap=cmap))
axs[0].label_outer()
axs[0].text(0.8, 0.95, '(a)', transform=axs[0].transAxes,
      fontsize=16, va='top', color='w')
axs[0].set_xticks([])
axs[0].set_yticks([])
axs[0].set_xlabel(r'$P(n,n´,l)$')
axs[0].set_ylabel(r'$atom_i(r)$')

data = mat_opt
images.append(axs[1].imshow(data, cmap=cmap))
axs[1].label_outer()
axs[1].text(0.8, 0.95, '(b)', transform=axs[1].transAxes,
      fontsize=16, va='top', color='w')
axs[1].set_xticks([])
axs[1].set_yticks([])
axs[1].set_xlabel(r'$P(n,n´,l)$')
axs[1].set_ylabel(r'$atom_i(r)$')

# Find the min and max of all colors for use in setting the color scale.
vmin = min(image.get_array().min() for image in images)
vmax = max(image.get_array().max() for image in images)
norm = colors.Normalize(vmin=vmin, vmax=vmax)
for im in images:
    im.set_norm(norm)

fig.colorbar(images[0], ax=axs, orientation='horizontal', fraction=.1)

def update(changed_image):
    for im in images:
        if (changed_image.get_cmap() != im.get_cmap()
                or changed_image.get_clim() != im.get_clim()):
            im.set_cmap(changed_image.get_cmap())
            im.set_clim(changed_image.get_clim())


for im in images:
    im.callbacksSM.connect('changed', update)


plt.show()